In [ ]:
import numpy as np
import sys
#INSERT THE PATH TO THE PROJECT FOLDER HERE
PROJECT_PATH = "/home/mariusvaardal/AAMAS_project/AAMAS_project"
sys.path.append(PROJECT_PATH)
import utils.utils as utils
import math
from pettingzoo.mpe import simple_tag_v3

from agent_types.CoordinatingAgent import CoordinatingAgent
from agent_types.GreedyAgent import GreedyAgent
from agent_types.RandomAgent import RandomAgent
from agent_types.AvoidingAgent import AvoidingAgent
from agent_types.AvoidingNearestAdversaryAgent import AvoidingNearestAdversaryAgent
from agent_types.ImmobileAgent import ImmobileAgent
from agent_types.RLAgent_2_adv import RLAgent2


NUM_EPISODES = 50
NUM_GOOD = 1
NUM_ADVERSARIES = 2
NUM_LANDMARKS = 0
MAX_CYCLES = 200
RENDER_MODE = 'human'

env = simple_tag_v3.parallel_env(num_good=NUM_GOOD, num_adversaries=NUM_ADVERSARIES, num_obstacles=NUM_LANDMARKS, max_cycles=MAX_CYCLES, continuous_actions=False, render_mode=RENDER_MODE)
env.reset()

coord_agents = [RLAgent2(name, NUM_ADVERSARIES, NUM_LANDMARKS) for name in env.agents if name != "agent_0"]
coord_agents.append(AvoidingAgent("agent_0", NUM_ADVERSARIES, NUM_LANDMARKS))

episode_rewards = []
for episode in range(1, NUM_EPISODES + 1):
    observations, infos = env.reset()
    for agent in coord_agents:
        agent.observations = observations[agent.name]
        agent.update_observed_agent_positions()
        
    episode_reward = 0
    while env.agents:
        actions = {}
        for agent in coord_agents:
            action = agent.get_action()
            actions[agent.name] = action if action != None else 0
        observations, rewards, terminations, truncations, infos = env.step(actions)
        for agent in coord_agents:
            agent.observations = observations[agent.name]
            agent.update_observed_agent_positions()
        episode_reward += utils.get_timestep_reward(rewards)
    episode_rewards.append(episode_reward)
print(f"Avg : {sum(episode_rewards) / len(episode_rewards)}. For {len(episode_rewards)} number of episodes")

env.close()








